In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm

import torch

def evaluate(model, test_x, test_y):
    model.eval()  # Set the model to evaluation mode
    acc = 0.0
    total_samples = len(test_x)

    with torch.no_grad():
        for imgs, labels in zip(test_x, test_y):
            # Convert the data to PyTorch tensors
            imgs = torch.tensor(imgs).view(1, -1)
            labels = torch.tensor(labels)

            # Forward pass to get predictions
            outputs = model(imgs)

            # Get the predicted class
            _, predicted = torch.max(outputs, 1)

            # Update accuracy
            acc += (predicted == labels).sum().item()

    # Calculate and return the accuracy
    accuracy = (acc / total_samples) * 100
    return round(accuracy, 2)


class MLP3(nn.Module):
    def __init__(self, input_shape=(32,32,3), hidden_layers_neuron_list=[200, 200], num_classes=2):
        super(MLP3, self).__init__()

        self.input_shape = input_shape
        self.num_classes = num_classes
        self.hidden_layers_neuron_list = hidden_layers_neuron_list

        # Create layers
        self.flatten = nn.Flatten()
        # self.fc1 = nn.Linear(input_shape[0] * input_shape[1]*input_shape[2], hidden_layers_neuron_list[0])
        self.fc1 = nn.Linear(3072, 200)
        self.fc2 = nn.Linear(hidden_layers_neuron_list[0], hidden_layers_neuron_list[1])
        self.fc3 = nn.Linear(hidden_layers_neuron_list[1], num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Train:
    def __init__(self, optimizer, loss_fn, prior_weights=None, lambda_=0.1):
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.prior_weights = prior_weights
        self.lambda_ = lambda_

    def train(self, model, epochs, train_task, fisher_matrix=None, test_tasks=None):
        if test_tasks:
            test_acc = [[] for _ in test_tasks]
        else:
            test_acc = None
        for epoch in tqdm(range(epochs)):
            for batch in train_task:
                X, y = batch
                y =  y.type(torch.LongTensor)
                self.optimizer.zero_grad()
                pred = model(X)
                loss = self.loss_fn(pred, y)
                if fisher_matrix is not None:
                    loss += self.compute_penalty_loss(model, fisher_matrix)
                loss.backward()
                self.optimizer.step()
            if test_acc:
                for i in range(len(test_tasks)):
                  for inputs,labels in validloader:
                    test_acc[i].append(evaluate(model, inputs, labels))
            return test_acc

    def compute_penalty_loss(self, model, fisher_matrix):
        penalty = 0.
        for u, v, w in zip(fisher_matrix, model.parameters(), self.prior_weights):
            penalty += torch.sum(u * (v - w)**2)
        return 0.5 * self.lambda_ * penalty

class EWC:
    def __init__(self, prior_model, data_samples, num_sample=30):
        self.prior_model = prior_model
        self.prior_weights = [param.data for param in prior_model.parameters()]
        self.num_sample = num_sample
        self.data_samples = data_samples
        self.fisher_matrix = self.compute_fisher()

    def compute_fisher(self):
        weights = self.prior_weights
        fisher_accum = [torch.zeros_like(w) for w in weights]
        for j in tqdm(range(self.num_sample)):
            idx = np.random.randint(self.data_samples.shape[0])
            data = torch.tensor(self.data_samples[idx]).view(1, -1)
            logits = self.prior_model(data)
            log_probs = torch.nn.functional.log_softmax(logits, dim=1)
            grads = torch.autograd.grad(log_probs, weights, create_graph=True)
            grads = [g**2 for g in grads]
            fisher_accum = [acc + grad for acc, grad in zip(fisher_accum, grads)]
        fisher_accum = [acc / self.num_sample for acc in fisher_accum]
        return fisher_accum

    def get_fisher(self):
        return self.fisher_matrix

In [ ]:
from torchvision import datasets
import os
import torchvision.transforms as transforms
import torch

batch_size = 8
transform = transforms.Compose([transforms.Resize(32),
                                transforms.CenterCrop(32),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485,0.456,0.405], [0.229, 0.224, 0.225])])

data_dir = '/content/drive/MyDrive/IIIT/Various_datasets/hymenoptera_data-20231108T102316Z-001/hymenoptera_data'
train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform)


trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
validloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

train_data_size = len(train_data)
val_data_size = len(val_data)
print(train_data_size)
print(val_data_size)

class_names = train_data.classes
print(class_names)

254
153
['ants', 'bees']


In [ ]:
#Define the model
model = MLP3(input_shape=(32,32,3), hidden_layers_neuron_list=[200, 200], num_classes=2)

In [ ]:
epochs = 10
lambda_ = 0.1
lr = 0.00001
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.BinaryCrossEntropyLoss()

AttributeError: ignored

In [ ]:
# Create a training instance
trainer = Train(optimizer, loss_fn, prior_weights=None, lambda_=0.1)

# Train the model on task A
acc_prior_A = trainer.train(model, epochs, trainloader, fisher_matrix=None, test_tasks=[validloader])

# Save the trained model
torch.save(model.state_dict(), 'Hymenoptera.pth')

# Print the accuracy for Task A
print('[INFO] Task A Original (SGD): {}'.format(acc_prior_A[-1]))

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-1-d5dc7c2bbbab>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs = torch.tensor(imgs).view(1, -1)
<ipython-input-1-d5dc7c2bbbab>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)
  0%|          | 0/10 [00:02<?, ?it/s]

[INFO] Task A Original (SGD): [25.0, 50.0, 62.5, 62.5, 62.5, 62.5, 87.5, 75.0, 37.5, 50.0, 75.0, 62.5, 75.0, 62.5, 50.0, 75.0, 25.0, 50.0, 75.0, 0.0]


NotImplementedError: ignored

In [ ]:
 dataset = dataset.map(lambda x: tf.py_function(func=map_example, inp=[x['height'], x['width'], x['image_raw'], x['label']], Tout=(tf.int64, tf.int64)))
 dataset = dataset.map(lambda x: map_example(x['height'], x['width'], x['image_raw'], x['label']))
    # dataset = dataset.batch(batch_size=FLAGS.batch_size)